$\Large\textbf{Lab 7.} \large\textbf{Exercise 2.}$



In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from timeit import default_timer as timer
np.random.seed(1000)

In [2]:
#Now we will define a function which will compute and return the function value 
def evalf(x, n, lamda):  
  #Input: x is a numpy array of size n 
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments 
  #assert len(x) == n #do not allow arbitrary size arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  #x = np.reshape(x, [2,1])
  return 0.5 * (np.linalg.norm(np.subtract(np.matmul(A,x) , y)))**2 + lamda * (np.linalg.norm(x)**2) * 0.5

In [3]:
#Now we will define a function which will compute and return the gradient value as a numpy array 
def evalg(x, n, lamda):
  assert type(x) is np.ndarray 
  #assert len(x) == n
  #x = np.reshape(x, [2,1])
  return np.add(np.matmul(A.T, np.subtract(np.matmul(A, x), y)), lamda * x)

In [4]:
def evalh(x, n, lamda):
  assert type(x) is np.ndarray  #do not allow arbitrary type arguments 
  #assert len(x) == n #do not allow arbitrary size arguments
  #x = np.reshape(x, [2,1])
  return np.add(np.matmul(A.T,A), lamda * np.identity(n))

In [5]:
def compute_B_k(s, y, B_k, n, lamda):
  assert type(s) is np.ndarray #do not allow arbitrary type arguments 
  assert type(y) is np.ndarray #do not allow arbitrary type arguments 
  assert type(B_k) is np.ndarray #do not allow arbitrary type arguments 
  assert len(s) == n and len(y) == n #do not allow arbitrary size arguments 
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  I = np.identity(n)
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [6]:
#line search type 
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

In [7]:
def compute_D_k(x,n, lamda):
  assert type(x) is np.ndarray
  assert len(x) == n
  mat = evalh(x, n, lamda)
  return np.linalg.inv(mat)

In [8]:
def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma, B_k, lamda, n): #add appropriate arguments to the function 
  assert type(x) is np.ndarray
  assert type(gradf) is np.ndarray
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p =  -gradf
  D_k = B_k
  #implement the backtracking line search
  while evalf(x + alpha*np.matmul(D_k,p), n, lamda) > evalf(x, n, lamda) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p))):
    alpha = alpha * rho
  return alpha  

In [9]:
def find_minimizer_Newtonmethod(start_x, n, tol, line_search_type , lamda, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0 
  x = start_x
  g_x = evalg(x,n, lamda)
  h_x = evalh(x,n, lamda)
  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  #print('iter:',k,  ' f(x):', evalf(x,n), ' gradient norm:', np.linalg.norm(g_x))
  step_length = 1.
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #implement the Newton's method here
    D_k = np.linalg.inv(evalh(x, n, lamda))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x, g_x, alpha_start, rho, gamma, D_k, lamda, n)
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x, n, lamda) #compute gradient at new point
  return x

In [10]:
def find_minimizer_BFGS_scaling(start_x, n, tol, line_search_type, lamda, *args):
  #Input: start_x is a numpy array of size n, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray #do not allow arbitrary type arguments 
  assert len(start_x) == n #do not allow arbitrary size arguments 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x,n, lamda)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  B_k = np.identity(n)
  x_old = x
  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old, g_x, alpha_start, rho, gamma, B_k, lamda, n)
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x))) 
    g_x = evalg(x_new,n, lamda)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new,n, lamda), evalg(x_old,n, lamda)) 
    B_k = compute_B_k(s, y, B_k, n, lamda)
    x_old = x_new
    k += 1
  return x_new

In [ ]:
#Code for Newton method
my_tol = my_tol= 1e-5
alpha = 0.9
rho = 0.5
gamma = 0.5
import numpy as np
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1) #random noise
#For each value of dimension in the ds array, we will check the behavior of Newton method
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  xorig = np.ones((d,1))
  my_start_x = np.zeros((d,1))
  y = np.dot(A,xorig) + eps
  start = timeit.default_timer()
  #call Newton method with A,y,lambda and obtain the optimal solution x_opt
  x_opt = find_minimizer_Newtonmethod(my_start_x, d, my_tol, BACKTRACKING_LINE_SEARCH, lambda_reg, alpha, rho, gamma)
  #x_opt = Newton(A,y,lambda_reg)
  newtontime = timeit.default_timer() - start #time is in seconds
  #print the total time and the L2 norm difference || x_opt - xorig|| for Newton method
  diff = np.subtract(x_opt, xorig)
  print("\n\nd:", d, "\nTotal time:", newtontime, "\nL2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_opt), y))**2,"\nL2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)



d: 1000 
Total time: 1.5028203279998706 
L2 norm difference ||Ax* - y||_2^2:  2.1230188630378149e-07 
L2 norm difference || x_opt - xorig||:  817.7202986460508


d: 5000 
Total time: 120.09765258000016 
L2 norm difference ||Ax* - y||_2^2:  4.7444073837853636e-08 
L2 norm difference || x_opt - xorig||:  4774.913359905506


d: 10000 
Total time: 969.8989088329995 
L2 norm difference ||Ax* - y||_2^2:  1.752138900998938e-08 
L2 norm difference || x_opt - xorig||:  9825.861490427424


$\large\textbf{Question 1.}$

$\large\textbf{Answer 1.}$

In the previous exercise, the hessian matrix was $A^TA$ but by adding reguralizer term, it is becoming $A^TA+\lambda I$. One cannot assure the invertibility of the matrix $A^TA$, so it is possible that inverse of hessian matrix doesn't exist for previous exercise but the matrix $A^TA+\lambda I$ is always invertible because $A^TA$ is symmetric and by adding positive diagonal elements to symmetric matrix, We assure the positiveness of eigenvalues.Therefore for $\lambda > 0 $, the inverse of hessian matrix always exist for regularized function in this exercise.

In [ ]:
#Code for BFGS method
print("Using BFGS Method")
my_tol = my_tol= 1e-5
alpha = 0.9
rho = 0.5
gamma = 0.5
import numpy as np
import timeit
np.random.seed(1000) #for repeatability
N = 200
ds = [1000, 5000, 10000, 20000, 25000, 50000, 100000, 200000, 500000, 1000000]
lambda_reg = 0.001
eps = np.random.randn(N,1) #random noise
#For each value of dimension in the ds array, we will check the behavior of BFGS method
for i in range(np.size(ds)):
  d=ds[i]
  A = np.random.randn(N,d)
  my_start_x = np.zeros((d,1))
  xorig = np.ones((d,1))
  y = np.dot(A,xorig) + eps
  #start = timeit.default_timer()
  #call Newton method with A,y,lambda and obtain the optimal solution x_opt

  #x_opt = Newton(A,y,lambda_reg)
  #newtontime = timeit.default_timer() - start #time is in seconds
  #print the total time, ||Ax_opt-y||^2 and the L2 norm difference || x_opt - xorig||^2  for Newton method
  start = timeit.default_timer()
  #call BFGS method with A,y,lambda and obtain the optimal solution x_opt_bfgs
  x_star_bfgs = find_minimizer_BFGS_scaling(my_start_x, d, my_tol, BACKTRACKING_LINE_SEARCH, lambda_reg, alpha, rho, gamma)

  #x_opt_bfgs = BFGS(A,y,lambda_reg)
  bfgstime = timeit.default_timer() - start #time is in seconds
  #print the total time, ||Ax_opt_bfgs-y||^2 and the L2 norm difference || x_opt_bfgs -  xorig||^2 for BFGS method
  diff = np.subtract(x_star_bfgs, xorig)
  print("\n\nd:", d, "\nTotal time:", bfgstime, "\nL2 norm difference ||Ax* - y||_2^2: ", np.linalg.norm(np.subtract(np.matmul(A, x_star_bfgs), y))**2,"\nL2 norm difference || x_opt - xorig||: ", np.linalg.norm(diff)**2)

**Question 1**


Newton's Method

Dimention of A | Total Time |  $||Ax^* - y||_2^2$   |  $||x^* - x_{orig}||_2^2$ 
:----:|:---:|:---:|:---:
(200, 1000) | 1.6312558310000895 | 2.1230188630378149e-07 | 817.7202986460508
(200, 5000) | 122.61732197399988 | 4.7444073837853636e-08   | 4774.913359905506
(200, 10000) | 979.5976196650001  |1.752138900998938e-08 | 9825.861490427424


BFGS Method

Dimention of A | Total Time |  $||Ax^* - y||_2^2$   |  $||x^* - x_{orig}||_2^2$ 
:----:|:---:|:---:|:---:
(200, 1000) | 21.26525222400005  | 0.0021220539229979173   | 817.7202119606469
(200, 5000) | 112.11423496499992  | 0.00047397442649197916  | 4774.913361486921
(200, 10000) | 906.3106474900001  |0.00017517588089034528 | 9825.861491699694

**Question 2**

The failure occures for newton's method after size of A = (200, 10000)
The failure occures for BFGS method after size of A = (200, 10000)